# Data Scientist Professional Practical Exam Submission

**Use this template to write up your summary for submission. Code in Python or R needs to be included.**


## 📝 Task List

Your written report should include both code, output and written text summaries of the following:
- Data Validation:   
  - Describe validation and cleaning steps for every column in the data 
- Exploratory Analysis:  
  - Include two different graphics showing single variables only to demonstrate the characteristics of data  
  - Include at least one graphic showing two or more variables to represent the relationship between features
  - Describe your findings
- Model Development
  - Include your reasons for selecting the models you use as well as a statement of the problem type
  - Code to fit the baseline and comparison models
- Model Evaluation
  - Describe the performance of the two models based on an appropriate metric
- Business Metrics
  - Define a way to compare your model performance to the business
  - Describe how your models perform using this approach
- Final summary including recommendations that the business should undertake

*Start writing report here..*

In [6]:
import pandas as pd

# Load the data
df = pd.read_csv('recipe_site_traffic_2212.csv')

# Inspect the data
print(df.head())

# Handling missing values
df.fillna(value=pd.np.nan, inplace=True)

# Convert 'servings' to numeric data type
df['servings'] = pd.to_numeric(df['servings'], errors='coerce')

# Handle 'high_traffic' column (assuming NA means not high traffic)
df['high_traffic'].fillna('Not High', inplace=True)

# Inspect the cleaned data
print(df.head())

# Save the cleaned data
df.to_csv('cleaned_recipe_site_traffic.csv', index=False)


   recipe  calories  carbohydrate  ...   category  servings high_traffic
0       1       NaN           NaN  ...       Pork         6         High
1       2     35.48         38.56  ...     Potato         4         High
2       3    914.28         42.68  ...  Breakfast         1          NaN
3       4     97.03         30.56  ...  Beverages         4         High
4       5     27.05          1.85  ...  Beverages         4          NaN

[5 rows x 8 columns]
   recipe  calories  carbohydrate  ...   category  servings high_traffic
0       1       NaN           NaN  ...       Pork       6.0         High
1       2     35.48         38.56  ...     Potato       4.0         High
2       3    914.28         42.68  ...  Breakfast       1.0     Not High
3       4     97.03         30.56  ...  Beverages       4.0         High
4       5     27.05          1.85  ...  Beverages       4.0     Not High

[5 rows x 8 columns]


## ✅ When you have finished...
-  Publish your Workspace using the option on the left
-  Check the published version of your report:
	-  Can you see everything you want us to grade?
    -  Are all the graphics visible?
-  Review the grading rubric. Have you included everything that will be graded?
-  Head back to the [Certification Dashboard](https://app.datacamp.com/certification) to submit your practical exam report and record your presentation